In [1]:
import pandas
import requests
import spotipy
import string
import re

from spotipy.oauth2 import SpotifyClientCredentials
from genderize import Genderize

"""
# Not really working
# From 'Song for Bob Dylan', extracted the name 'Song' :(
import spacy
import spacy.cli
spacy.cli.download('en_core_web_lg')
en = spacy.load('en_core_web_lg')

"""


"\n# Not really working\n# From 'Song for Bob Dylan', extracted the name 'Song' :(\nimport spacy\nimport spacy.cli\nspacy.cli.download('en_core_web_lg')\nen = spacy.load('en_core_web_lg')\n\n"

In [2]:
with open('vars.env', 'r') as f:
    my_id = f.readline().rstrip('\n')
    my_secret = f.readline().rstrip('\n')
    playlistURI = f.readline().rstrip('\n')
    owner = f.readline().rstrip('\n')

client_credentials_manager = SpotifyClientCredentials(client_id=my_id, client_secret=my_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlistID = playlistURI[-22:]

In [3]:
results = sp.user_playlist(owner, playlist_id=playlistID, fields='tracks, next')['tracks']

songs =  results['items']
while results['next']:
    results = sp.next(results)
    songs.extend(results['items'])

idSet = {track['added_by']['id'] for track in songs}    
us = {(id, sp.user(id)['display_name']) for id in idSet}
us = dict(us)
us['rmb101'] = 'rmb101'

print(len(songs))


241


In [4]:
# Make the dataframe by pulling out the song name, artist and person who added it
table = [(track['track']['name'], track['track']['artists'][0]['name'], us[track['added_by']['id']]) 
         for track in songs]
df = pandas.DataFrame(table, columns = ['Song', 'Artist', 'Added by'])

# Get rid of anything after a hyphen, as this is all '2012 remastered' kinda text
# also remove all apostrophe s as this seems to confuse the names
# and get rid of digits
df['Song'].replace('\d| - .*|\'s','', regex = True, inplace = True)

# set up regex to kill off remaining punctuation
punclist = string.punctuation
punc = re.compile('[%s]' % re.escape(punclist))

# bring in list of common words (ie not names) to remove
# alsoa list of known problem words that aren't names
words = pandas.read_csv('top5000.csv')
wordlist = words['Top words'].tolist()
wordlist.remove('sue')
blocklist = ['rigby', 'marmalade', 'runaround', 'dont', 'hasty', 'mambo', 'holland', 'speedy', 'lind', 'stars', 'o', 'ode', 'punk', 'rocker']
wordlist.extend(blocklist)


# function to remove the punctuation and common names
# returns as list because I'm going to run genderize over it as a list
def namesOnly(row):
    text = punc.sub('', row['Song'].lower()).split()
    newTitle = [word for word in text if word not in wordlist]
    return newTitle

df['ShortTitle'] = df.apply(namesOnly, axis = 1)


In [5]:
# Function to try to assign a gender to each word in each shortened song title
def gender(song):
    response = Genderize().get(song['ShortTitle'])
    return response
    #return [(word['gender'], word['probability']) for word in response] 

df['GenderData'] = df.apply(gender, axis = 1)

df['GenderData'].head()
# I think speed this up by getting a single set of names to run it on first
# then join the results up with the list - so I don't do each name more than once

0    [{'name': 'billie', 'gender': 'female', 'proba...
1    [{'name': 'ziggy', 'gender': 'male', 'probabil...
2    [{'name': 'diana', 'gender': 'female', 'probab...
3    [{'name': 'johnny', 'gender': 'male', 'probabi...
4    [{'name': 'jolene', 'gender': 'female', 'proba...
Name: GenderData, dtype: object

In [6]:
# Make this part of main genderising function once happy with it
def tidy(song):
    words = [[word['gender'], word['probability']] for word in song['GenderData'] if word['gender']]
    guesses = [word[0] for word in words if word[1] > 0.7] 
    if len(set(guesses)) == 1:
        return guesses[0]
    elif not guesses:
        return ''
    else:
        return guesses

df['Guesses'] = df.apply(tidy, axis = 1)

df.head()

,Song,Artist,Added by,ShortTitle,GenderData,Guesses
0,Billie Jean,Michael Jackson,Diana Be,"[billie, jean]","[{'name': 'billie', 'gender': 'female', 'proba...",female
1,Ziggy Stardust,David Bowie,Tom Williams,"[ziggy, stardust]","[{'name': 'ziggy', 'gender': 'male', 'probabil...",male
2,Dirty Diana,Michael Jackson,Diana Be,[diana],"[{'name': 'diana', 'gender': 'female', 'probab...",female
3,Johnny Too Bad,John Martyn,rjdw500,[johnny],"[{'name': 'johnny', 'gender': 'male', 'probabi...",male
4,Jolene,Dolly Parton,danielnissenbaum,[jolene],"[{'name': 'jolene', 'gender': 'female', 'proba...",female


In [7]:
df[['ShortTitle', 'Guesses']].loc[~df['Guesses'].astype(str).isin(['male', 'female']) ]

#df[['Song','Guesses']].to_csv('Attempt1.csv')
#dirty
#block
#rigby
#theme
#dear
#ill dance
#holland
#dont

,ShortTitle,Guesses
17,"[eleanor, rigby]","[female, male]"
48,[chunli],
61,[marmalade],
70,[],
76,[runaround],
92,"[jenny, dont, hasty]","[female, male, male]"
94,[],
95,[cleopatras],
99,"[mambo, 5]",
102,"[mary, jane, holland]","[female, female, male]"


In [ ]:
# Manually correct the ones it couldn't work out 

In [ ]:
# now for artists (argh)


# Not really working

def entity(text):
    nameList = [name for name in en(text['Song']).ents if name.label_ == 'PERSON']
    if nameList:
        return nameList[0]
    else:
        return None

df['SongNames'] = df.apply(entity, axis = 1)
df.head(50)

In [9]:
df[70:80]

,Song,Artist,Added by,ShortTitle,GenderData,Guesses
70,Name,The Goo Goo Dolls,Diana Be,[],[],
71,Kim,Joy Again,Adam Smith,[kim],"[{'name': 'kim', 'gender': 'female', 'probabil...",female
72,Polly,Nirvana,Diana Be,[polly],"[{'name': 'polly', 'gender': 'female', 'probab...",female
73,Danny Nedelko,IDLES,Adam Smith,"[danny, nedelko]","[{'name': 'danny', 'gender': 'male', 'probabil...",male
74,Put Your Dukes Up John,Arctic Monkeys,sarahhuntington,"[dukes, john]","[{'name': 'dukes', 'gender': None}, {'name': '...",male
75,Hercules,Aaron Neville,Adam Smith,[hercules],"[{'name': 'hercules', 'gender': 'male', 'proba...",male
76,Runaround Sue,Dion,sarahhuntington,[runaround],"[{'name': 'runaround', 'gender': None}]",
77,Maggie May,Rod Stewart,Chelle Cole,[maggie],"[{'name': 'maggie', 'gender': 'female', 'proba...",female
78,Jimmy Mack,Martha Reeves & The Vandellas,Adam Smith,"[jimmy, mack]","[{'name': 'jimmy', 'gender': 'male', 'probabil...",male
79,Belle,Richard White,Chelle Cole,[belle],"[{'name': 'belle', 'gender': 'female', 'probab...",female
